Perhaps mark each pdf with a code denoting the bank type, so this file only has to do a simple string parse to know what to do. i.e. 'JohnDoeCAP.pdf', 'MarkAnthonyNED.pdf', etc. 

Sometimes Description is long and overflows. I cut it off, but should we keep it all?

In [1]:
import math 
import tabula as tb
from tabula import read_pdf

#constant conversion factor, per adobe
#pdf measurer pdf unit = 1/72 in
cf =72  

#all measurements in format [top,left,bottom,width]

#page counter
from PyPDF2 import PdfFileReader
def count_pdf_pages(file):
    with open(file, 'rb') as pdf:
        page_count = PdfFileReader(pdf).getNumPages()
    return page_count

In [42]:
%%time
import pandas as pd
#CAPITEC
#complete

#capitec measurements
#first page:
page_1_box = [3.77, 0.4, 9.8, 8.5,]
#middle pages:
middle_pages_box = [0.3, 0.4, 10.2, 8.5]

file = "Capitec.pdf"
page_count = count_pdf_pages(file)

#convert boxes
for i in range(0, len(page_1_box)):
    page_1_box[i] *= cf
    
for i in range(0, len(middle_pages_box)):
    middle_pages_box[i] *= cf

#get first page
page_1_table = tb.read_pdf(file, pages=1,area=[page_1_box],output_format="dataframe", stream=True)
page_1_df = page_1_table[0].drop(labels='Posting Date', axis=1)

#clean first page
page_1_df = page_1_df.rename(columns={'Transaction': 'Transaction Date'})
page_1_df = page_1_df.drop(index=0)
page_1_df = page_1_df.dropna(subset=['Transaction Date','Balance (R)'])

#append first page
total_df = pd.DataFrame(columns=page_1_df.columns)
total_df = pd.concat([total_df, page_1_df])


for page in range(2, page_count+1):

    #get middle page
    middle_pages_table = tb.read_pdf(file, pages=page, area=[middle_pages_box],
                                 output_format="dataframe", stream=True, pandas_options={'header': None})
    middle_df = middle_pages_table[0]

    #clean middle page
    middle_df = middle_df.dropna(axis='columns', how='all')
    middle_df.columns = page_1_df.columns


    middle_df = middle_df.dropna(subset=['Transaction Date','Balance (R)'])
    middle_df['Transaction Date'] = middle_df['Transaction Date'].map(lambda x: str(x).split()[-1])

    #append to total df
    total_df = pd.concat([total_df, middle_df])





total_df


Wall time: 2.59 s


,Transaction Date,Description,Money In (R),Money Out (R),Balance (R)
1,30/08/2021,Branch Card Replacement Fee,NaN,-70.0,-57.64
2,30/08/2021,Cash Deposit Cash Dep Dnr Boksbur Dnr Boksbur...,100.0,NaN,42.36
4,30/08/2021,Cash Deposit Fee (Notes),NaN,-1.3,41.06
5,30/08/2021,Banking App Prepaid Purchase MTN,NaN,-15.0,26.06
6,30/08/2021,Prepaid Mobile Purchase Fee,NaN,-0.5,25.56
...,...,...,...,...,...
9,04/10/2021,ATM Cash Withdrawal Sparkatm Bp Roodekop,NaN,-100.0,338.9
11,04/10/2021,ATM Cash Withdrawal Fee,nk,-7.5,331.4
12,04/10/2021,Payment Received: Rtc 05dtnty7sh Tk Capitec B2...,a,NaN,531.4
13,04/10/2021,Banking App Transfer to Rainy Days (1709362276...,NaN,-195.0,336.4


In [36]:
import pandas as pd
#Mercantile Bank
#can these be longer than the one provided??

            #all measurements in format [top,left,bottom,width]

#mercantile measurements
#first page:
page_1_box = [4.2, 0.4, 8.9, 7.7,]
#middle pages:
middle_pages_box = [2.4, 0.4, 10.1, 7.7]

file = "Mercantile Bank.pdf"
page_count = count_pdf_pages(file)

#convert boxes
for i in range(0, len(page_1_box)):
    page_1_box[i] *= cf
    
for i in range(0, len(middle_pages_box)):
    middle_pages_box[i] *= cf

#get first page
page_1_table = tb.read_pdf(file, pages=1,area=[page_1_box],output_format="dataframe", stream=True)
page_1_df = page_1_table[0].drop(labels='Post', axis=1)

#page_1_df
#clean first page
page_1_df = page_1_df.rename(columns={'Trans.': 'Transaction Date'})
page_1_df = page_1_df.drop(index=0)
######page_1_df = page_1_df.dropna(subset=['Transaction Date','Balance (R)'])

#append first page
total_df = pd.DataFrame(columns=page_1_df.columns)
total_df = pd.concat([total_df, page_1_df])


for page in range(2, page_count+1):

    #get middle page
    middle_pages_table = tb.read_pdf(file, pages=page, area=[middle_pages_box],
                                 output_format="dataframe", stream=True, pandas_options={'header': None})
    middle_df = middle_pages_table[0]

    #clean middle page
    middle_df = middle_df.drop(labels=0, axis=1)
    middle_df = middle_df.drop(index=[0,1])
    middle_df = middle_df.dropna(axis='columns', how='all')
    middle_df.columns = page_1_df.columns
    middle_df = middle_df.dropna(subset=['Transaction Date','Balance'])

    #append to total df
    total_df = pd.concat([total_df, middle_df])





total_df


,Transaction Date,Description,Reference,Fees,Amount,Balance
1,NaN,Balance brought forward,NaN,NaN,NaN,+2 320.16
2,NaN,Interest Rate @ 0.000,NaN,NaN,NaN,NaN
3,06/01/22,POS Local Purchase,0000000000002723 BUCCLEUCH SERVICE ST,-5.00,-200.00,+2 115.16
4,NaN,NaN,JOHANNESBURG AUTH ID 260106,NaN,NaN,NaN
5,07/01/22,POS Local Purchase,0000000000002723 Yoco Sandton AUTH ID,-5.00,-81.00,+2 029.16
6,NaN,NaN,178515,NaN,NaN,NaN
7,07/01/22,ATM Withdrawal,NaN,-9.75,-1 000.00,+1 019.41
8,07/01/22,POS Local Purchase,0000000000002723 MCD Midridge (0344),-5.00,-181.30,+833.11
9,NaN,NaN,MIDRANDLICAB AUTH ID 297461,NaN,NaN,NaN
10,07/01/22,Fuel Purchase,0000000000002723 Cltx Buccleuch SANDTON,-5.00,-100.00,+728.11


In [3]:
import pandas as pd

df1 = pd.DataFrame([['a', 1,2,3,4], ['b', 2],['a', 1], ['b', 2],['a', 1]],
                   columns=range(0,5))
df1

,0,1,2,3,4
0,a,1,2.0,3.0,4.0
1,b,2,NaN,NaN,NaN
2,a,1,NaN,NaN,NaN
3,b,2,NaN,NaN,NaN
4,a,1,NaN,NaN,NaN
